In [1]:
import warnings
warnings.filterwarnings("ignore")
import shutil
import os
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pickle
from sklearn.manifold import TSNE
from sklearn import preprocessing
import pandas as pd
from multiprocessing import Process# this is used for multithreading
import multiprocessing 
import random as r
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import log_loss
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import MiniBatchKMeans

In [2]:
features = ['duration','protocol_type','service','flag','src_bytes','dst_bytes','land','wrong_fragment','urgent','hot','num_failed_logins','logged_in','num_compromised','root_shell','su_attempted',
'num_root','num_file_creations','num_shells','num_access_files','num_outbound_cmds',
'is_host_login',
'is_guest_login',
'count',
'srv_count',
'serror_rate',
'srv_serror_rate',
'rerror_rate',
'srv_rerror_rate',
'same_srv_rate',
'diff_srv_rate',
'srv_diff_host_rate',
'dst_host_count',
'dst_host_srv_count',
'dst_host_same_srv_rate',
'dst_host_diff_srv_rate',
'dst_host_same_src_port_rate',
'dst_host_srv_diff_host_rate',
'dst_host_serror_rate',
'dst_host_srv_serror_rate',
'dst_host_rerror_rate',
'dst_host_srv_rerror_rate',
'intrusion_type']

In [4]:
data = pd.read_csv('kddcup.data_10_percent_corrected', names=features, header=None)
data.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,intrusion_type
0,0,tcp,http,SF,181,5450,0,0,0,0,...,9,1.0,0.0,0.11,0.0,0.0,0.0,0.0,0.0,normal.
1,0,tcp,http,SF,239,486,0,0,0,0,...,19,1.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,normal.
2,0,tcp,http,SF,235,1337,0,0,0,0,...,29,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal.
3,0,tcp,http,SF,219,1337,0,0,0,0,...,39,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal.
4,0,tcp,http,SF,217,2032,0,0,0,0,...,49,1.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0,normal.


In [5]:
print('The no of data points are:',data.shape[0])
print('='*40)
print('The no of features are:',data.shape[1])
print('='*40)
print('Some of the features are:',features[:10])

The no of data points are: 494021
The no of features are: 42
Some of the features are: ['duration', 'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes', 'land', 'wrong_fragment', 'urgent', 'hot']


In [6]:
output = data['intrusion_type'].values
labels = set(output)

In [7]:
print('The different type of output labels are:',labels)
print('='*100)
print('No. of different output labels are:', len(labels))

The different type of output labels are: {'multihop.', 'normal.', 'perl.', 'warezmaster.', 'smurf.', 'teardrop.', 'imap.', 'neptune.', 'back.', 'nmap.', 'buffer_overflow.', 'phf.', 'land.', 'ftp_write.', 'warezclient.', 'spy.', 'ipsweep.', 'guess_passwd.', 'loadmodule.', 'rootkit.', 'portsweep.', 'pod.', 'satan.'}
No. of different output labels are: 23


In [9]:
duplicateRowsDF = data[data.duplicated()]

In [10]:
duplicateRowsDF.head(5)

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,intrusion_type
2500,0,tcp,http,SF,307,60990,0,0,0,0,...,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,normal.
2864,0,tcp,http,SF,351,11485,0,0,0,0,...,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,normal.
2977,0,tcp,http,SF,506,2419,0,0,0,0,...,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,normal.
3003,0,tcp,http,SF,319,1374,0,0,0,0,...,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,normal.
3130,0,tcp,http,SF,234,1212,0,0,0,0,...,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,normal.


In [11]:
data.drop_duplicates(subset=features, keep='first', inplace=True)
data.shape

(145586, 42)

In [12]:
data.to_pickle('data.pkl')
data = pd.read_pickle('data.pkl')

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(data.drop('intrusion_type', axis=1), data['intrusion_type'], stratify=data['intrusion_type'], test_size=0.25)

In [14]:
print('Train data')
print(X_train.shape)
print(Y_train.shape)
print('='*20)
print('Test data')
print(X_test.shape)
print(Y_test.shape)

Train data
(109189, 41)
(109189,)
Test data
(36397, 41)
(36397,)


In [15]:
protocol = list(X_train['protocol_type'].values)
protocol = list(set(protocol))
print('Protocol types are:', protocol)

Protocol types are: ['tcp', 'udp', 'icmp']


In [16]:
from sklearn.feature_extraction.text import CountVectorizer
one_hot = CountVectorizer(vocabulary=protocol, binary=True)
train_protocol = one_hot.fit_transform(X_train['protocol_type'].values)
test_protocol = one_hot.transform(X_test['protocol_type'].values)

In [17]:
print(train_protocol[1].toarray())
train_protocol.shape

[[1 0 0]]


(109189, 3)

In [18]:
service = list(X_train['service'].values)
service = list(set(service))
print('Service types are:\n', service)

Service types are:
 ['auth', 'pop_2', 'bgp', 'tim_i', 'uucp', 'klogin', 'ftp_data', 'supdup', 'kshell', 'http', 'sunrpc', 'name', 'urp_i', 'ntp_u', 'smtp', 'uucp_path', 'remote_job', 'daytime', 'pop_3', 'csnet_ns', 'urh_i', 'discard', 'echo', 'eco_i', 'ssh', 'ldap', 'telnet', 'gopher', 'netbios_ssn', 'http_443', 'netbios_dgm', 'nnsp', 'netbios_ns', 'shell', 'printer', 'courier', 'ftp', 'link', 'rje', 'mtp', 'login', 'X11', 'efs', 'ecr_i', 'domain', 'domain_u', 'vmnet', 'ctf', 'iso_tsap', 'sql_net', 'hostnames', 'time', 'Z39_50', 'whois', 'private', 'systat', 'nntp', 'IRC', 'netstat', 'imap4', 'other', 'exec', 'finger']


In [19]:
from sklearn.feature_extraction.text import CountVectorizer
one_hot = CountVectorizer(vocabulary=service, binary=True)
train_service = one_hot.fit_transform(X_train['service'].values)
test_service = one_hot.transform(X_test['service'].values)

In [20]:
print(train_service[100].toarray())

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]]


In [21]:
train_service.shape

(109189, 63)

In [22]:
flag = list(X_train['flag'].values)
flag = list(set(flag))
print('flag types are:', flag)

flag types are: ['S2', 'REJ', 'RSTOS0', 'OTH', 'S3', 'RSTO', 'SH', 'S1', 'S0', 'RSTR', 'SF']


In [23]:
from sklearn.feature_extraction.text import CountVectorizer
one_hot = CountVectorizer(binary=True)
one_hot.fit(X_train['flag'].values)
train_flag = one_hot.transform(X_train['flag'].values)
test_flag = one_hot.transform(X_test['flag'].values)

In [24]:
print(test_flag[3000].toarray())
train_flag.shape

[[0 0 0 0 0 0 0 0 0 1 0]]


(109189, 11)

In [25]:
X_train.drop(['protocol_type','service','flag'], axis=1, inplace=True)
X_test.drop(['protocol_type','service','flag'], axis=1, inplace=True)

In [26]:
def feature_scaling(X_train, X_test, feature_name):
    
    '''
    This function performs standardisation on the features
    '''
    
    from sklearn.preprocessing import StandardScaler

    scaler = StandardScaler()
    scaler1 = scaler.fit_transform(X_train[feature_name].values.reshape(-1,1))
    scaler2 = scaler.transform(X_test[feature_name].values.reshape(-1,1))
    
    return scaler1, scaler2

In [27]:
duration1, duration2 = feature_scaling(X_train, X_test, 'duration')

In [28]:
print(duration1[1])

[-0.10910614]


In [29]:
src_bytes1, src_bytes2 = feature_scaling(X_train, X_test, 'src_bytes')

In [30]:
print(src_bytes1[1])

[-0.00427531]


In [31]:
dst_bytes1, dst_bytes2 = feature_scaling(X_train, X_test, 'dst_bytes')

In [32]:
print(dst_bytes1[1])

[-0.01498087]


In [33]:
wrong_fragment1, wrong_fragment2 = feature_scaling(X_train, X_test, 'wrong_fragment')

In [34]:
print(wrong_fragment1[1])

[-0.08439045]


In [35]:
urgent1, urgent2 = feature_scaling(X_train, X_test, 'urgent')

In [36]:
print(urgent1[1])

[-0.00546978]


In [37]:
hot1, hot2 = feature_scaling(X_train, X_test, 'hot')

In [38]:
print(hot1[1])

[-0.07069875]


In [39]:
num_failed_logins1, num_failed_logins2 = feature_scaling(X_train, X_test, 'num_failed_logins')

In [40]:
print(num_failed_logins1[1])

[-0.01805418]


In [41]:
num_compromised1, num_compromised2 = feature_scaling(X_train, X_test, 'num_compromised')

In [42]:
num_compromised1[1]

array([-0.00774397])

In [43]:
root_shell1, root_shell2 = feature_scaling(X_train, X_test, 'root_shell')

In [44]:
root_shell1[1]

array([-0.01890255])

In [45]:
su_attempted1, su_attempted2 = feature_scaling(X_train, X_test, 'su_attempted')

In [46]:
su_attempted1[1]

array([-0.00647062])

In [47]:
num_root1, num_root2 = feature_scaling(X_train, X_test, 'num_root')

In [48]:
num_root1[1]

array([-0.00993182])

In [49]:
num_file_creations1, num_file_creations2 = feature_scaling(X_train, X_test, 'num_file_creations')

In [50]:
num_file_creations1[1]

array([-0.02129663])

In [51]:
num_shells1, num_shells2 = feature_scaling(X_train, X_test, 'num_shells')

In [52]:
num_shells1[1]

array([-0.01859328])

In [53]:
num_access_files1, num_access_files2 = feature_scaling(X_train, X_test, 'num_access_files')

In [54]:
num_access_files1[1]

array([-0.04966817])

In [55]:
data['num_outbound_cmds'].value_counts()

num_outbound_cmds
0    145586
Name: count, dtype: int64

In [56]:
srv_count1, srv_count2 = feature_scaling(X_train, X_test, 'srv_count')

In [57]:
srv_count1[1]

array([-0.32568762])

In [58]:
serror_rate1, serror_rate2 = feature_scaling(X_train, X_test, 'serror_rate')

In [59]:
serror_rate1[1]

array([-0.64443784])

In [60]:
srv_serror_rate1, srv_serror_rate2 = feature_scaling(X_train, X_test, 'srv_serror_rate')

In [61]:
srv_serror_rate1[1]

array([-0.64385987])

In [62]:
rerror_rate1, rerror_rate2 = feature_scaling(X_train, X_test, 'rerror_rate')

In [63]:
rerror_rate1[1]

array([-0.35002801])

In [64]:
srv_rerror_rate1, srv_rerror_rate2 = feature_scaling(X_train, X_test, 'srv_rerror_rate')

In [65]:
srv_rerror_rate1[1]

array([-0.35039265])

In [66]:
same_srv_rate1, same_srv_rate2 = feature_scaling(X_train, X_test, 'same_srv_rate')

In [67]:
same_srv_rate1[1]

array([0.77193037])

In [68]:
diff_srv_rate1, diff_srv_rate2 = feature_scaling(X_train, X_test, 'diff_srv_rate')

In [69]:
diff_srv_rate1[1]

array([-0.33660747])

In [70]:
srv_diff_host_rate1, srv_diff_host_rate2 = feature_scaling(X_train, X_test, 'srv_diff_host_rate')

In [71]:
srv_diff_host_rate1[1]

array([-0.3856542])

In [72]:
dst_host_count1, dst_host_count2 = feature_scaling(X_train, X_test, 'dst_host_count')

In [73]:
dst_host_count1[1]

array([0.74106411])

In [74]:
dst_host_srv_count1, dst_host_srv_count2 = feature_scaling(X_train, X_test, 'dst_host_srv_count')

In [75]:
dst_host_srv_count1[1]

array([1.09085178])

In [76]:
dst_host_same_srv_rate1, dst_host_same_srv_rate2= feature_scaling(X_train, X_test, 'dst_host_same_srv_rate')

In [77]:
dst_host_diff_srv_rate1, dst_host_diff_srv_rate2 = feature_scaling(X_train, X_test, 'dst_host_diff_srv_rate')

In [78]:
dst_host_same_src_port_rate1, dst_host_same_src_port_rate2 = feature_scaling(X_train, X_test, 'dst_host_same_src_port_rate')

In [79]:
dst_host_same_src_port_rate1[1]

array([-0.38636499])

In [80]:
dst_host_srv_diff_host_rate1, dst_host_srv_diff_host_rate2 = feature_scaling(X_train, X_test, 'dst_host_srv_diff_host_rate')

In [81]:
dst_host_srv_diff_host_rate1[1]

array([-0.3134176])

In [82]:
dst_host_serror_rate1, dst_host_serror_rate2 = feature_scaling(X_train, X_test, 'dst_host_serror_rate')

In [83]:
dst_host_serror_rate1[1]

array([-0.64541986])

In [84]:
dst_host_srv_serror_rate1, dst_host_srv_serror_rate2 = feature_scaling(X_train, X_test, 'dst_host_srv_serror_rate')

In [85]:
dst_host_rerror_rate1, dst_host_rerror_rate2 = feature_scaling(X_train, X_test, 'dst_host_rerror_rate')

In [86]:
dst_host_srv_rerror_rate1, dst_host_srv_rerror_rate2 = feature_scaling(X_train, X_test, 'dst_host_srv_rerror_rate')

In [87]:
dst_host_srv_rerror_rate1[1]

array([-0.35293996])

In [88]:
num_failed_logins1, num_failed_logins2 = feature_scaling(X_train, X_test, 'num_failed_logins')

In [89]:
land1, land2 = np.array([X_train['land'].values]), np.array([X_test['land'].values])

In [90]:
is_host_login1, is_host_login2 = np.array([X_train['is_host_login'].values]), np.array([X_test['is_host_login'].values])

In [91]:
is_guest_login1, is_guest_login2 = np.array([X_train['is_guest_login'].values]), np.array([X_test['is_guest_login'].values])

In [92]:
logged_in1, logged_in2 = np.array([X_train['logged_in'].values]), np.array([X_test['logged_in'].values])

In [93]:
logged_in1.shape

(1, 109189)

In [94]:
count1, count2 = feature_scaling(X_train, X_test, 'count')

In [95]:
count1[1]

array([-0.71140569])

In [96]:
dst_host_diff_srv_rate1, dst_host_diff_srv_rate2 = feature_scaling(X_train, X_test, 'dst_host_diff_srv_rate')

In [97]:
dst_host_diff_srv_rate1[1]

array([-0.41723695])

In [98]:
from scipy.sparse import hstack

In [99]:
X_train_1 = hstack((duration1, train_protocol, train_service, train_flag, src_bytes1,
       dst_bytes1, land1.T, wrong_fragment1, urgent1, hot1,
       num_failed_logins1, logged_in1.T, num_compromised1, root_shell1,
       su_attempted1, num_root1, num_file_creations1, num_shells1,
       num_access_files1, is_host_login1.T,
       is_guest_login1.T, count1, srv_count1, serror_rate1,
       srv_serror_rate1, rerror_rate1, srv_rerror_rate1, same_srv_rate1,
       diff_srv_rate1, srv_diff_host_rate1, dst_host_count1,
       dst_host_srv_count1, dst_host_same_srv_rate1,
       dst_host_diff_srv_rate1, dst_host_same_src_port_rate1,
       dst_host_srv_diff_host_rate1, dst_host_serror_rate1,
       dst_host_srv_serror_rate1, dst_host_rerror_rate1,
       dst_host_srv_rerror_rate1))

In [100]:
X_train_1.shape

(109189, 114)

In [101]:
X_test_1 = hstack((duration2, test_protocol, test_service, test_flag, src_bytes2,
       dst_bytes2, land2.T, wrong_fragment2, urgent2, hot2,
       num_failed_logins2, logged_in2.T, num_compromised2, root_shell2,
       su_attempted2, num_root2, num_file_creations2, num_shells2,
       num_access_files2, is_host_login2.T,
       is_guest_login2.T, count2, srv_count2, serror_rate2,
       srv_serror_rate2, rerror_rate2, srv_rerror_rate2, same_srv_rate2,
       diff_srv_rate2, srv_diff_host_rate2, dst_host_count2,
       dst_host_srv_count2, dst_host_same_srv_rate2,
       dst_host_diff_srv_rate2, dst_host_same_src_port_rate2,
       dst_host_srv_diff_host_rate2, dst_host_serror_rate2,
       dst_host_srv_serror_rate2, dst_host_rerror_rate2,
       dst_host_srv_rerror_rate2))

In [102]:
X_test_1.shape

(36397, 114)

In [103]:
import joblib
joblib.dump(X_train_1,'X_train_1.pkl')
joblib.dump(X_test_1,'X_test_1.pkl')
X_train_1 = joblib.load('X_train_1.pkl')
X_test_1 = joblib.load('X_test_1.pkl')

In [104]:
joblib.dump(Y_train,'Y_train.pkl')
joblib.dump(Y_test,'Y_test.pkl')
Y_train = joblib.load('Y_train.pkl')
Y_test = joblib.load('Y_test.pkl')

In [105]:
from sklearn.metrics import precision_score


In [106]:
def generate_random_individuals(population_size, num_features, min_features, max_features):
    individuals = np.zeros((population_size, num_features))
    for i in range(population_size):
        num_ones = np.random.randint(min_features, max_features+1)
        ones_indices = np.random.choice(num_features, num_ones, replace=False)
        individuals[i, ones_indices] = 1
    return individuals

In [155]:
# # 2.& 3.: Train model and get precision
# def train_model(x_train, x_test, y_train, y_test, predictor_names):
#     x_train = x_train.loc[:, predictor_names]
#     x_test = x_test.loc[:, predictor_names]
    
#     #Building the random forest model 
#     mdl = RandomForestClassifier(random_state=1) #Creating the Random Forest Classifier Model
#     mdl.fit(x_train, y_train)                    #Training the Model with x_train & y_train
#     y_hat = mdl.predict(x_test)                  #Predicting the x_test 
#     prec = precision_score(y_test, y_hat)        #Precision: The best value is 1 and the worst value is 0.
#     return prec

import scipy

from scipy.sparse import coo_matrix
from sklearn.ensemble import RandomForestClassifier

def train_model(x_train, x_test, y_train, y_test, predictor_names):
    if isinstance(x_train, scipy.sparse.coo_matrix):
        # If x_train is a sparse matrix, convert it to a dense format for RandomForestClassifier
        x_train = x_train.todense()
        x_test = x_test.todense()

    # Convert predictor_names to a list of column indices
    column_indices = [i for i, name in enumerate(x_train.columns) if name in predictor_names]

    # Use the list of column indices for slicing
    x_train = x_train[:, column_indices]
    x_test = x_test[:, column_indices]

    # Building the random forest model
    model = RandomForestClassifier()
    model.fit(x_train, y_train)

    # Evaluate the model
    accuracy = model.score(x_test, y_test)
    return accuracy


In [108]:
def choose_parents(population,accuracy,elite_percent):
    # Get elite of top 2 which doesn't mutate
    elite_num = int(round(((elite_percent*population.shape[0]) // 2) * 2))
    ind_ac = np.argsort(-accuracy)
    top_perc = ind_ac[:elite_num]
    elite_population = population[top_perc,:]   # We should keep this elite
    
    # Normalize accuracy to obtain weights for roulette wheel selection
    weight_norm = accuracy / accuracy.sum()     # calculate normalised weight from accuracy
    weight_comu = weight_norm.cumsum()          # calc cumulative weight from accuracy
   
    # Roulette wheel selection   
    num_parents_wo_elite = population.shape[0] - elite_num
    parents_wo_elite = np.empty([num_parents_wo_elite,population.shape[1]])
    for count in range(num_parents_wo_elite):
        b = weight_comu[-1]                         # current last element of weight_comu 
        rand_num = np.random.uniform(0,b)         # random foating-point number btw 0 and current max weight_comu
        
        indices = np.searchsorted(weight_comu,rand_num) # get indices of the number in weight_comu greater than rand_num
        parents_wo_elite[count,:] = population[indices, :]
        
    parents = np.concatenate((elite_population, parents_wo_elite), axis=0)  # Concatenate elite and parents_wo_elite to get all parents
    return parents

In [129]:
def one_point_crossover(parents,elite_percent,mutation_probability,min_features, max_features):
    elite_num = int(round(((elite_percent*population.shape[0]) // 2) * 2))
    crossover_population = np.zeros((parents.shape[0],parents.shape[1]))         # first two are elite
    crossover_population[0:elite_num,:] = parents[0:elite_num,:]
    
    for ii in range(int((parents.shape[0]-elite_num)/2)):
        n = 2*ii+elite_num                    # gives even number
        parents_couple = parents[n:n+2, :]    # comb of parents
        b2 = parents.shape[1]                 # num of features
        rand_n = np.random.randint(1, b2-1)   # generate rand number from 1 to num_of_features-1
        crossover_population[n,:] = np.concatenate([parents_couple[0, :rand_n], parents_couple[1, rand_n:]])
        crossover_population[n+1,:] = np.concatenate([parents_couple[1, :rand_n], parents_couple[0, rand_n:]])
        
    #check if every child has minimum number of features or all true values
    for kk in range(crossover_population.shape[0]):
        Sum = np.sum(crossover_population[kk,:])
        if Sum > max_features:
            # if the number of 1s is bigger than max number of features
            excess = int(Sum - max_features)
            indices = np.where(crossover_population[kk,:] == 1)[0]
            position1 = np.random.choice(indices, size=excess, replace=False)       
            crossover_population[kk, position1] = 0 # put 0s in random positions
        elif Sum < min_features:
            # if the number of 1s is smaller than min number of features
            missing = int(min_features - Sum)
            indices = np.where(crossover_population[kk,:] == 0)[0]
            position2 = np.random.choice(indices, size=missing, replace=False) 
            crossover_population[kk, position2] = 1 # put 1s in random positions

    # mutation
    child_row = crossover_population.shape[0]
    child_col = crossover_population.shape[1]
    num_mutations = round(child_row*child_col*mutation_probability)     
    for jj in range(num_mutations):
        ind_row = np.random.randint(0,child_row) # random number btw 0 and num of rows
        ind_col = np.random.randint(0,child_col) # random number btw 0 and num of colmns
        if (crossover_population[ind_row,ind_col] == 0 and 
            np.sum(crossover_population[ind_row,:]) < max_features):
            crossover_population[ind_row,ind_col] = 1
        elif (crossover_population[ind_row,ind_col] == 1 and 
              np.sum(crossover_population[ind_row,:]) >= min_features+1):
            crossover_population[ind_row,ind_col] = 0 
    
    return crossover_population

In [130]:
# Extract calssification target
target = data.loc[:, "intrusion_type"]
# Convert the target variable to a binary representation (0 or 1)
target = (target == "normal.").astype(int)
# Extract all other columns except 'diagnosis' as predictors
data_predictors = data.loc[:, data.columns != 'intrusion_type']
# Create a list of predictor names
predictor_names = data_predictors.columns

In [131]:
# Hyperparameters
num_features = data_predictor.shape[1]
min_features = 2            # minimal number of features in a subset of features
population_size = 8         # size of population (number of instances)
max_iterations = 8          # maximum number of iterations
elite_percent = 0.4         # percentage of elite population which doesn't mutate
mutation_probability = 0.2  # percentage of total genes that mutate
max_features = 4            # maximum number of features in a subset of features

In [156]:
# Main loop - gen 0
#1&2 generate population
population = generate_random_individuals(population_size, num_features, min_features, max_features)   
#3&4 train model
accuracy = np.zeros(population_size);   
predictor_names = data_predictor.columns
response_name = target.name                                                 
for i in range(population_size):
    predictor_names_i = predictor_names[population[i,:]==1]
    accuracy_i = train_model(X_train_1,X_test_1,Y_train,Y_test,predictor_names_i)
    accuracy[i] = accuracy_i
gen = 0
best_acc_i = np.zeros(max_iterations)    
best_acc_i[gen] = max(accuracy)     # keep best accuracy from 1st gen

AttributeError: 'matrix' object has no attribute 'columns'

In [151]:
# Main loop - gen > 0
gen =7
while gen < max_iterations-1:
    print('Begin iteration num {}/{}'.format(gen+2,max_iterations))
    gen += 1
    parents = choose_parents(population, accuracy, elite_percent)
    children = one_point_crossover(parents,elite_percent,mutation_probability,min_features,max_features)
    population = children
    for ind in range(population_size):
        predictor_names_ind = predictor_names[population[ind,:]==1]
        accuracy_ind = train_model(X_train_1,X_test_1,Y_train,Y_test,predictor_names_ind)
        accuracy[ind] = accuracy_ind
    best_acc_i[gen] = max(accuracy)
    
ind_max_acc = np.argmax(accuracy)
best_features = population[ind_max_acc,:]

In [152]:
# Plots - Post-processing
best_features_names = predictor_names[best_features==1]
plt.plot(range(1, max_iterations + 1), best_acc_i)
plt.xlabel('Number of generations')
plt.ylabel('Best accuracy')
plt.title('Genetic algorithm')
plt.show()

NameError: name 'best_acc_i' is not defined

==========================================================================================

In [158]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from random import randint
%matplotlib inline 
import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split
def split(df,label):
    X_tr, X_te, Y_tr, Y_te = train_test_split(df, label, test_size=0.25, random_state=42)
    return X_tr, X_te, Y_tr, Y_te

from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold, cross_val_score

classifiers = ['LinearSVM', 'RadialSVM', 
               'Logistic',  'RandomForest', 
               'AdaBoost',  'DecisionTree', 
               'KNeighbors','GradientBoosting']

models = [svm.SVC(kernel='linear'),
          svm.SVC(kernel='rbf'),
          LogisticRegression(max_iter = 1000),
          RandomForestClassifier(n_estimators=200, random_state=0),
          AdaBoostClassifier(random_state = 0),
          DecisionTreeClassifier(random_state=0),
          KNeighborsClassifier(),
          GradientBoostingClassifier(random_state=0)]


def acc_score(df,label):
    Score = pd.DataFrame({"Classifier":classifiers})
    j = 0
    acc = []
    X_train,X_test,Y_train,Y_test = split(df,label)
    for i in models:
        model = i
        model.fit(X_train,Y_train)
        predictions = model.predict(X_test)
        acc.append(accuracy_score(Y_test,predictions))
        j = j+1     
    Score["Accuracy"] = acc
    Score.sort_values(by="Accuracy", ascending=False,inplace = True)
    Score.reset_index(drop=True, inplace=True)
    return Score

def plot(score,x,y,c = "b"):
    gen = [1,2,3,4,5]
    plt.figure(figsize=(6,4))
    ax = sns.pointplot(x=gen, y=score,color = c )
    ax.set(xlabel="Generation", ylabel="Accuracy")
    ax.set(ylim=(x,y))


In [159]:
def initilization_of_population(size,n_feat):
    population = []
    for i in range(size):
        chromosome = np.ones(n_feat,dtype=np.bool)     
        chromosome[:int(0.3*n_feat)]=False             
        np.random.shuffle(chromosome)
        population.append(chromosome)
    return population


def fitness_score(population):
    scores = []
    for chromosome in population:
        logmodel.fit(X_train.iloc[:,chromosome],Y_train)         
        predictions = logmodel.predict(X_test.iloc[:,chromosome])
        scores.append(accuracy_score(Y_test,predictions))
    scores, population = np.array(scores), np.array(population) 
    inds = np.argsort(scores)                                    
    return list(scores[inds][::-1]), list(population[inds,:][::-1]) 


def selection(pop_after_fit,n_parents):
    population_nextgen = []
    for i in range(n_parents):
        population_nextgen.append(pop_after_fit[i])
    return population_nextgen


def crossover(pop_after_sel):
    pop_nextgen = pop_after_sel
    for i in range(0,len(pop_after_sel),2):
        new_par = []
        child_1 , child_2 = pop_nextgen[i] , pop_nextgen[i+1]
        new_par = np.concatenate((child_1[:len(child_1)//2],child_2[len(child_1)//2:]))
        pop_nextgen.append(new_par)
    return pop_nextgen


def mutation(pop_after_cross,mutation_rate,n_feat):   
    mutation_range = int(mutation_rate*n_feat)
    pop_next_gen = []
    for n in range(0,len(pop_after_cross)):
        chromo = pop_after_cross[n]
        rand_posi = [] 
        for i in range(0,mutation_range):
            pos = randint(0,n_feat-1)
            rand_posi.append(pos)
        for j in rand_posi:
            chromo[j] = not chromo[j]  
        pop_next_gen.append(chromo)
    return pop_next_gen

def generations(df,label,size,n_feat,n_parents,mutation_rate,n_gen,X_train,
                                   X_test, Y_train, Y_test):
    best_chromo= []
    best_score= []
    population_nextgen=initilization_of_population(size,n_feat)
    for i in range(n_gen):
        scores, pop_after_fit = fitness_score(population_nextgen)
        print('Best score in generation',i+1,':',scores[:1])  #2
        pop_after_sel = selection(pop_after_fit,n_parents)
        pop_after_cross = crossover(pop_after_sel)
        population_nextgen = mutation(pop_after_cross,mutation_rate,n_feat)
        best_chromo.append(pop_after_fit[0])
        best_score.append(scores[0])
    return best_chromo,best_score


In [168]:
from scipy.sparse import vstack

data_pcos = vstack([X_train_1, X_test_1])
label_pcos = np.hstack([Y_train, Y_test])


In [ ]:
# score4 = acc_score(data_pcos,label_pcos)
# score4

In [ ]:
logmodel = RandomForestClassifier(n_estimators=200, random_state=0)
X_train,X_test, Y_train, Y_test = split(data_pcos,label_pcos)
chromo_df_pcos,score_pcos=generations(data_pcos,label_pcos,size=80,n_feat=data_pcos.shape[1],n_parents=64,mutation_rate=0.20,n_gen=5,
                         X_train = X_train,X_test = X_test,Y_train = Y_train,Y_test = Y_test)